<a href="https://colab.research.google.com/github/pankajr141/experiments/blob/master/Reasoning/DataMining/datamining_frequent_itemset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd

dataset = [
    ['A', 'B', 'C', 'D', 'E'],
    ['A', 'B', 'C', 'F'],
    ['A', 'B', 'D', 'E'],
    ['A', 'B', 'C', 'D', 'E', 'F'],
    ['B', 'C', 'E', 'F']
]

In [ ]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,A,B,C,D,E,F
0,True,True,True,True,True,False
1,True,True,True,False,False,True
2,True,True,False,True,True,False
3,True,True,True,True,True,True
4,False,True,True,False,True,True


## Frequent Itemset Mining (FIM)

### Apriori

https://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/

In [ ]:
%%time
from mlxtend.frequent_patterns import apriori
apriori(df, min_support=0.6, use_colnames=True)

CPU times: user 10.6 ms, sys: 61 µs, total: 10.7 ms
Wall time: 11.3 ms


,support,itemsets
0,0.8,(A)
1,1.0,(B)
2,0.8,(C)
3,0.6,(D)
4,0.8,(E)
5,0.6,(F)
6,0.8,"(A, B)"
7,0.6,"(A, C)"
8,0.6,"(A, D)"
9,0.6,"(A, E)"


### FP-Growth

In [ ]:
%%time
from mlxtend.frequent_patterns import fpgrowth
fpgrowth(df, min_support=0.6, use_colnames=True)

CPU times: user 6.25 ms, sys: 0 ns, total: 6.25 ms
Wall time: 6.28 ms


,support,itemsets
0,1.0,(B)
1,0.8,(E)
2,0.8,(C)
3,0.8,(A)
4,0.6,(D)
5,0.6,(F)
6,0.8,"(B, E)"
7,0.8,"(B, C)"
8,0.6,"(C, E)"
9,0.6,"(B, C, E)"


### Eclat

In [ ]:
!pip install pyECLAT --quiet

In [ ]:
import numpy as np
df_eclat = pd.DataFrame(dataset)
df_eclat

,0,1,2,3,4,5
0,A,B,C,D,E,None
1,A,B,C,F,None,None
2,A,B,D,E,None,None
3,A,B,C,D,E,F
4,B,C,E,F,None,None


In [ ]:
from pyECLAT import ECLAT
eclat_instance = ECLAT(data=df_eclat, verbose=True) #verbose=True to see the loading bar

100%|██████████| 7/7 [00:00<00:00, 1780.48it/s]


In [ ]:
eclat_instance.df_bin

,A,F,E,B,D,C,None
0,1,0,1,1,1,1,0
1,1,1,0,1,0,1,0
2,1,0,1,1,1,0,0
3,1,1,1,1,1,1,0
4,0,1,1,1,0,1,0


In [ ]:
eclat_instance.uniq_

['A', 'F', 'E', 'B', 'D', 'C', None]

In [ ]:
get_ECLAT_indexes, get_ECLAT_supports = eclat_instance.fit(min_support=0.6,
                                                          min_combination=2,
                                                          max_combination=4,
                                                          separator=' & ',
                                                          verbose=True)
get_ECLAT_supports

Combination 2 by 2


15it [00:00, 92.23it/s]


Combination 3 by 3


20it [00:00, 126.72it/s]


Combination 4 by 4


15it [00:00, 141.84it/s]


{'A & E': 0.6,
 'A & B': 0.8,
 'A & D': 0.6,
 'A & C': 0.6,
 'F & B': 0.6,
 'F & C': 0.6,
 'E & B': 0.8,
 'E & D': 0.6,
 'E & C': 0.6,
 'B & D': 0.6,
 'B & C': 0.8,
 'A & E & B': 0.6,
 'A & E & D': 0.6,
 'A & B & D': 0.6,
 'A & B & C': 0.6,
 'F & B & C': 0.6,
 'E & B & D': 0.6,
 'E & B & C': 0.6,
 'A & E & B & D': 0.6}

In [ ]:
get_ECLAT_indexes

{'A & E': [0, 2, 3],
 'A & B': [0, 1, 2, 3],
 'A & D': [0, 2, 3],
 'A & C': [0, 1, 3],
 'F & B': [1, 3, 4],
 'F & C': [1, 3, 4],
 'E & B': [0, 2, 3, 4],
 'E & D': [0, 2, 3],
 'E & C': [0, 3, 4],
 'B & D': [0, 2, 3],
 'B & C': [0, 1, 3, 4],
 'A & E & B': [0, 2, 3],
 'A & E & D': [0, 2, 3],
 'A & B & D': [0, 2, 3],
 'A & B & C': [0, 1, 3],
 'F & B & C': [1, 3, 4],
 'E & B & D': [0, 2, 3],
 'E & B & C': [0, 3, 4],
 'A & E & B & D': [0, 2, 3]}

In [ ]:
#This function will search for all possible combinations that have support > 0.02 (in this case)
get_ECLAT_indexes_all, get_ECLAT_supports_all = eclat_instance.fit_all(min_support=0.6,
                                                                      separator=' & ',
                                                                      verbose=True)
get_ECLAT_supports_all

Combination 1 by 1


6it [00:00, 103.21it/s]


Combination 2 by 2


15it [00:00, 114.73it/s]


Combination 3 by 3


20it [00:00, 92.29it/s] 


Combination 4 by 4


15it [00:00, 96.70it/s] 

Stopping in combination 4 by 4. All supports = 0.


{'A': 0.8,
 'F': 0.6,
 'E': 0.8,
 'B': 1.0,
 'D': 0.6,
 'C': 0.8,
 'A & E': 0.6,
 'A & B': 0.8,
 'A & D': 0.6,
 'A & C': 0.6,
 'F & B': 0.6,
 'F & C': 0.6,
 'E & B': 0.8,
 'E & D': 0.6,
 'E & C': 0.6,
 'B & D': 0.6,
 'B & C': 0.8,
 'A & E & B': 0.6,
 'A & E & D': 0.6,
 'A & B & D': 0.6,
 'A & B & C': 0.6,
 'F & B & C': 0.6,
 'E & B & D': 0.6,
 'E & B & C': 0.6,
 'A & E & B & D': 0.6}

## Maximal Frequent Itemset Mining (MFIM)


In [ ]:
%%time
from mlxtend.frequent_patterns import fpmax
fpmax(df, min_support=0.4, use_colnames=True)

CPU times: user 5.07 ms, sys: 6 µs, total: 5.08 ms
Wall time: 6.75 ms


,support,itemsets
0,0.4,"(A, E, B, D, C)"
1,0.4,"(F, A, C, B)"
2,0.4,"(F, B, C, E)"


## Closed Frequent Itemset Mining (CFIM)

In [ ]:
%%time
from mlxtend.frequent_patterns import fpgrowth
frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
closed_itemsets = frequent_itemsets[frequent_itemsets['support'] == frequent_itemsets['support']]
frequent_itemsets.shape, closed_itemsets.shape

CPU times: user 5.91 ms, sys: 896 µs, total: 6.8 ms
Wall time: 11.1 ms


((25, 2), (25, 2))

## Generalized Sequential Pattern Mining (GSPM)

### Prefixspan

In [ ]:
!pip install prefixspan --quiet

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done


In [ ]:
from prefixspan import PrefixSpan

# Apply PrefixSpan algorithm for sequential pattern mining with constraints
min_support = 2
patterns = PrefixSpan(dataset).frequent(min_support)

# Display the discovered sequential patterns
print("Frequent Sequential Patterns:")
for pattern in patterns:
    print(pattern)

Frequent Sequential Patterns:
(4, ['A'])
(4, ['A', 'B'])
(3, ['A', 'B', 'C'])
(2, ['A', 'B', 'C', 'D'])
(2, ['A', 'B', 'C', 'D', 'E'])
(2, ['A', 'B', 'C', 'E'])
(2, ['A', 'B', 'C', 'F'])
(3, ['A', 'B', 'D'])
(3, ['A', 'B', 'D', 'E'])
(3, ['A', 'B', 'E'])
(2, ['A', 'B', 'F'])
(3, ['A', 'C'])
(2, ['A', 'C', 'D'])
(2, ['A', 'C', 'D', 'E'])
(2, ['A', 'C', 'E'])
(2, ['A', 'C', 'F'])
(3, ['A', 'D'])
(3, ['A', 'D', 'E'])
(3, ['A', 'E'])
(2, ['A', 'F'])
(5, ['B'])
(4, ['B', 'C'])
(2, ['B', 'C', 'D'])
(2, ['B', 'C', 'D', 'E'])
(3, ['B', 'C', 'E'])
(2, ['B', 'C', 'E', 'F'])
(3, ['B', 'C', 'F'])
(3, ['B', 'D'])
(3, ['B', 'D', 'E'])
(4, ['B', 'E'])
(2, ['B', 'E', 'F'])
(3, ['B', 'F'])
(4, ['C'])
(2, ['C', 'D'])
(2, ['C', 'D', 'E'])
(3, ['C', 'E'])
(2, ['C', 'E', 'F'])
(3, ['C', 'F'])
(3, ['D'])
(3, ['D', 'E'])
(4, ['E'])
(2, ['E', 'F'])
(3, ['F'])


## Mining frequent subgraphs

### gSpan

https://github.com/betterenvi/gSpan/tree/master

In [ ]:
!pip install gspan_mining --quiet

In [ ]:
from gspan_mining import gSpan

# Simulate a graph dataset
graph_dataset = [
    [(1, 2), (2, 3), (3, 4)],
    [(1, 2), (2, 3), (3, 4), (4, 5)],
    [(1, 2), (2, 3), (3, 4), (4, 6)],
    # ... (additional graph instances)
]

# Apply gSpan algorithm for mining frequent subgraphs
min_support = 2
gspan = gSpan(graph_dataset, min_support=min_support)
frequent_subgraphs = gspan.run()

# Display the discovered frequent subgraphs
print("Frequent Subgraphs:")
for subgraph in frequent_subgraphs:
    print(subgraph)

TypeError: expected str, bytes or os.PathLike object, not list